<div style="text-align: center;">
    <h1>Géolocalisation des entreprises de la base Sirene</h1>
</div>



Pour l'instant, les entreprises sont localisées par leur codeCommuneInsee, on les situe donc soit au milieu de la commune soit sur un point aléatoire dans la commune. On va tenter d'affiner cette localisation.

## 1. En enlevant les entreprises situées outre mer

## 2. En assignant à chaque entreprise une position (lat,lon) la plus fiable possible


Pour cela on va utiliser le dataset [*Géolocalisation des établissements du répertoire SIRENE-pour les études statistiques*](https://www.data.gouv.fr/fr/datasets/geolocalisation-des-etablissements-du-repertoire-sirene-pour-les-etudes-statistiques/#/resources) proposé par l'INSEE. Il propose une localisation (lat,lon) pour chaque entreprise avec différents niveaux de confiance. La documentation relative à ce jeu de données est disponible [ici](https://www.data.gouv.fr/fr/datasets/r/9f6c2157-3c89-4e8d-8473-9894348c84cb).



**L'objectif de ce notebook est d'ajouter une localisation à chaque établissement** du fichier d'entreprises que l'on a constitué ('./Données nationales/RegistreNationalEtablissementsActifsRneSirene.parquet).


On prend le parti pris suivant pour la suite : **Une entreprise sera décomptée si au moins une partie de la commune dans laquelle elle se situe est dans la zone de chalandise**.
La localisation de l'entreprise telle qu'on la considère ici n'est donc *pas un moyen de filtrer les entreprises* mais plutôt de donner des informations supplémentaires.

Ces informations seront ajoutées au fichier de base qui contiendra donc les variable suivantes :

- siret
- nomCommercial 
- adresseEtablissement : l'adresse en toute lettre
- codeInseecommune
- codeApe
- diffusionCommerciale
- localisation : (lat,lon) ou "nan"



In [1]:
import pandas as pd 
import folium 
import numpy as np
import geopandas as gpd
import random
from shapely.geometry import Point, Polygon
from tqdm.notebook import tqdm


/Users/basiledesjuzeur/Documents/Cartographie/venv_zone_de_chalandise/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# 1. Téléchargement et lecture du fichier

In [49]:
%%time 

# paramètres
url = "https://www.data.gouv.fr/fr/datasets/r/ba6a4e4c-aac6-4764-bbd2-f80ae345afc5"
id_doc = url.split('/')[-1]
usecols = ["siret","qualite_xy","x_longitude","y_latitude"]
dtypes = {"siret":np.int64,"qualite_xy":np.int8,"x_longitude":np.float64,"y_latitude":np.float64}


df = pd.read_csv(url,compression='zip',sep=';',usecols=usecols,dtype=dtypes)

CPU times: user 28.8 s, sys: 2.05 s, total: 30.8 s
Wall time: 1min 22s


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34513137 entries, 0 to 34513136
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   siret        int64  
 1   qualite_xy   int8   
 2   y_latitude   float64
 3   x_longitude  float64
dtypes: float64(2), int64(1), int8(1)
memory usage: 822.9 MB


# 2. Jointure avec le fichier Registre

In [51]:
# paramètres
path = "../Données nationales/RegistreNationalEtablissementsActifsRneSirene.parquet"
cols = ['siret', 'nomCommercial', 'adresseEtablissement', 'codeInseecommune',
       'codeApe', 'diffusionCommerciale']
dtypes = {'siret':np.int64,'nomCommercial':str,'adresseEtablissement':str,'codeInseecommune':str,
          'diffusionCommerciale':bool,'codeApe':str}

df_reg = pd.read_parquet("../Données nationales/RegistreNationalEtablissementsActifsRneSirene.parquet")

# fill na with 0 in numeric columns
df_reg = df_reg.fillna(0)

# change dtype
df_reg = df_reg.astype(dtypes)


Quelques infos avant de faire la jointure

In [52]:
nb_entreprises_loca = df.shape[0]
nb_entreprises_nation = df_reg.shape[0]

print("Nombre d'entreprises localisées dans le fichier de l'insee : {:,}".format(nb_entreprises_loca))
print("Nombre d'entreprises nationales dans le fichier concaténé : {:,}".format(nb_entreprises_nation))

Nombre d'entreprises localisées dans le fichier de l'insee : 34,513,137
Nombre d'entreprises nationales dans le fichier concaténé : 18,319,399


Pourquoi ce nombre ? (*de prime abord il devrait y avoir plus d'établissements dans le fichier concaténé aka. le registre*)

- La base Sirene que nous avions téléchargé faisait 37 768 658 lignes , mais seulement 15 millions d'établissements actifs

In [53]:

nb_in_df_and_reg = df.siret.isin(df_reg.siret).sum()
nb_in_reg_not_df = df_reg.siret[~df_reg.siret.isin(df.siret)].count()
nb_in_df_not_reg = df.siret[~df.siret.isin(df_reg.siret)].count()

print("Nombre d'entreprises localisées dans le fichier de l'insee et dans le fichier concaténé : {:,}".format(nb_in_df_and_reg))
print("Nombre d'entreprises localisées dans le fichier de l'insee mais pas dans le fichier concaténé : {:,}".format(nb_in_df_not_reg))
print("Nombre d'entreprises dans le fichier concaténé mais pas dans le fichier de l'insee : {:,}".format(nb_in_reg_not_df))

Nombre d'entreprises localisées dans le fichier de l'insee et dans le fichier concaténé : 16,630,088
Nombre d'entreprises localisées dans le fichier de l'insee mais pas dans le fichier concaténé : 17,883,049
Nombre d'entreprises dans le fichier concaténé mais pas dans le fichier de l'insee : 1,689,311


Jointure des deux dataframes

In [54]:
join = pd.merge(df,df_reg,how='right',on='siret').reset_index(drop=True)

On alège la mémoire en enlevant les dataframes inutilisées

In [55]:
del df
del df_reg

# 3. Suppression des entreprises hors de France métropolitaine


On part de la liste des codes communes des communes des outre-mer fournie par l'INSEE [ici](https://www.insee.fr/fr/information/6800675).

In [56]:
url = "https://www.insee.fr/fr/statistiques/fichier/6800675/v_commune_comer_2023.csv"

df_communes = pd.read_csv(url,sep=',')


df_communes

,COM_COMER,TNCC,NCC,NCCENR,LIBELLE,NATURE_ZONAGE,COMER,LIBELLE_COMER
0,97501,0,MIQUELON LANGLADE,Miquelon-Langlade,Miquelon-Langlade,COM,975,Saint-Pierre-et-Miquelon
1,97502,0,SAINT PIERRE,Saint-Pierre,Saint-Pierre,COM,975,Saint-Pierre-et-Miquelon
2,97701,0,SAINT BARTHELEMY,Saint-Barthélemy,Saint-Barthélemy,COM,977,Saint-Barthélemy
3,97801,0,SAINT MARTIN,Saint-Martin,Saint-Martin,COM,978,Saint-Martin
4,98411,4,ILES SAINT PAUL ET AMSTERDAM,Îles Saint-Paul et Amsterdam,Îles Saint-Paul et Amsterdam,DIS,984,Terres australes et antarctiques françaises
...,...,...,...,...,...,...,...,...
89,98830,0,TOUHO,Touho,Touho,COM,988,Nouvelle-Calédonie
90,98831,0,VOH,Voh,Voh,COM,988,Nouvelle-Calédonie
91,98832,0,YATE,Yaté,Yaté,COM,988,Nouvelle-Calédonie
92,98833,0,KOUAOUA,Kouaoua,Kouaoua,COM,988,Nouvelle-Calédonie


Quels sont les départements concernés ?

In [57]:
list = join[join.codeInseecommune.isin(df_communes["COM_COMER"].astype(str))].codeInseecommune.unique()

list = [str(x)[:2] for x in list]

list = set(list)

print("Nombre de départements concernés : {}".format(len(list)))
print("Liste des départements concernés : {}".format(list))

Nombre de départements concernés : 2
Liste des départements concernés : {'98', '97'}


In [58]:
# on ne garde que les communes qui ne sont pas dans les DOM-TOM (pas dans la liste list)

join = join[~join.codeInseecommune.astype(str).str[:2].isin(list)].reset_index(drop=True)

Sauvegarde du fichier

In [67]:
join.to_parquet("../Données nationales/RegistreNationalEtablissementsActifsRneSirene.parquet")

# 4. Complétion des données de localisation manquantes

Comme dit lors de l'introduction, on a envie de pouvoir signaler à l'utilisateur final que l'adresse n'est pas sure.

On va donc donner un indice de confiance plus lisible avec la correspondance suivantes (**voir documentation officielle INSEE**):

| Code | Type de Voie      | Détails                                    | Score |
|------|-------------------|--------------------------------------------|-------|
| 11    | Voie Sûre         | Numéro trouvé                               | 5     |
| 12   | Voie Sûre         | Position aléatoire dans la voie             | 4     |
| 21   | Voie probable     | Numéro trouvé                               | 3     |
| 22   | Voie probable     | Position aléatoire dans la voie             | 2     |
| 33   | Voie inconnue      | Position aléatoire dans la commune          | 1     |
| NaN  | Inconnu            | Position non renseignée dans le document de l'INSEE, localisation aléatoire dans la commune | 0   |



In [59]:
# change confiance
def give_confidence_score(x):
    
    correspondance = {11:5,12:4,21:3,22:2,33:1,0:0}

    return correspondance[x]

join.fillna(0,inplace=True)
join['confiance'] = join.qualite_xy.apply(give_confidence_score)



Pour les établissments qui correspondent au dernier cas de figure (position non renseignée dans le document de l'INSEE on attribue une position aléatoire dans la commune).

Pour cela on va s'aider du document 'populationLocalisationCommunes.parquet' constitué dans [ce notebook](../Notebooks/5.%20Localisation%20des%20communes.ipynb).



In [60]:
df_loc = gpd.read_parquet("../Données nationales/populationLocalisationCommunes.parquet")

On crée une fonction qui donne une position aléatoire dans une commune

In [44]:
def random_point_in_geometry(poly):

    if isinstance(poly, Polygon):

        minx, miny, maxx, maxy = poly.bounds
        while True:
            p = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
            if poly.contains(p):
                return (p.x,p.y )
            
    else:

        return None
        
# test de la fonction
geom_paris = df_loc[df_loc.codeCommune == '75101'].geometry.values[0]

x,y = random_point_in_geometry(geom_paris)

# affichage d'une carte folium  
m = folium.Map(location=[48.856578, 2.351828], zoom_start=12)
folium.GeoJson(geom_paris).add_to(m)

folium.Marker([y,x]).add_to(m)

m

On va maintenant appliquer cela à tous les points :
- on crée avec la geometry correspondant au code commune 
- on crée une colonne qui assigne un point aléatoire


1. Lecture du fichier

In [45]:
df = pd.read_parquet("../Données nationales/RegistreNationalEtablissementsActifsRneSirene.parquet")

2. Sélection des entreprises dont la localisation n'est pas renseignée, et ajout des colonnes néecessaires

In [ ]:
# filter on (df.confiance == 5) and (df.qualite_xy == 0)]
temp = df[(df.confiance == 0) & (df.qualite_xy == 0)]

df_loc = gpd.read_parquet("../Données nationales/populationLocalisationCommunes.parquet")[['codeCommune','geometryCommune']]

# join entre temp et df_loc keep 
temp = pd.merge(temp,df_loc,how='left',left_on='codeInseecommune',right_on='codeCommune')


# création d'une colonne avec une localisation aléatoire dans la commune
tqdm.pandas()
temp["random_point"] = temp.geometryCommune.progress_apply(random_point_in_geometry)

# création de deux colonnes avec les coordonnées x et y
temp[['x_longitude','y_latitude']] = pd.DataFrame(temp.random_point.tolist(), index=temp.index)

3. Remplacement des valeurs dans la df originelle

In [47]:
temp = temp.drop(columns=['random_point','geometryCommune'])

# replace elements in df with elements in temp
df.loc[temp.index] = temp


# sauvegarde du fichier
df.to_parquet("../Données nationales/RegistreNationalEtablissementsActifsRneSirene.parquet")


# 5. Formattage final du fichier

On formatte le fichier pour qu'il puisse être directement utilisé pour la carte. 
On lui ajoutes les colonnes suivantes :

- 'nom' : le nom de l'entrepise, à défaut son siret
- 'adresse' : l'adresse de l'entreprise, à défaut le code postal de la ville

In [10]:
df = pd.read_parquet("../Données nationales/RegistreNationalEtablissementsActifsRneSirene.parquet")



1. Le nom

In [11]:
def clean_nomCommercial(x):
    
    if x != "0":
        return x
    
    else:
        return "n.s. cf. Pappers"

df["nomCommercial"] = df.nomCommercial.apply(clean_nomCommercial)


2. L'adresse

On utilse ce jeu de données de correspondance OpenDataSoft ([ici](https://public.opendatasoft.com/explore/dataset/correspondance-code-insee-code-postal/table/)).

In [12]:
url = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/correspondance-code-insee-code-postal/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
usecols = ["Code INSEE","Code Postal","Commune"]
dtypes = {"Code INSEE":str,"Code Postal":str,"Commune":str}


df_corresp = pd.read_csv(url,sep=';',usecols=usecols,dtype=dtypes)

In [13]:
temp = df[df.adresseEtablissement == "0"]

temp = pd.merge(temp,df_corresp,how='left',left_on='codeInseecommune',right_on='Code INSEE')

temp["adresseEtablissement"] = temp["Code Postal"] + " " + temp["Commune"]

In [14]:
df.loc[temp.index] = temp

In [21]:
df.to_parquet("../Données nationales/RegistreNationalEtablissementsActifsRneSirene.parquet")